# Procesando datos de turismo

In [7]:
# 4_proceso_etl.py
from pathlib import Path
import pandas as pd
import numpy as np
from dateutil import parser

# Rutas
data_csv = (Path.cwd() / "../data/visitantes.csv").resolve()
db_dir   = (Path.cwd() / "../database").resolve()
db_dir.mkdir(parents=True, exist_ok=True)
out_csv  = db_dir / "visitantes_clean.csv"

if not data_csv.exists():
    raise FileNotFoundError(f"No existe {data_csv}. Ejecuta 3_generar_data_turismo.py primero.")

df = pd.read_csv(data_csv)
orig = len(df)

# Normalización de strings y NA/duplicados
df = df.applymap(lambda v: v.strip() if isinstance(v, str) else v)
df.replace({"": np.nan, "NA": np.nan, "N/A": np.nan}, inplace=True)
df.drop_duplicates(inplace=True)

# Completar faltantes razonables
df["medio_transporte"] = df.get("medio_transporte", np.nan).fillna("Desconocido")
df["destino"] = df.get("destino", np.nan).fillna("SinDestino")
df["region"]  = df.get("region",  np.nan).fillna("SinRegion")

# Edad → num; mediana por país; default=30
df["edad"] = pd.to_numeric(df.get("edad"), errors="coerce")
med_x_pais = df.groupby("pais_origen")["edad"].median()
def fill_edad(r):
    if pd.isna(r["edad"]):
        m = med_x_pais.get(r["pais_origen"], np.nan)
        return int(m) if not pd.isna(m) else 30
    return int(r["edad"])
df["edad"] = df.apply(fill_edad, axis=1)

# Fecha visita
def safe_date(x):
    if pd.isna(x): return pd.NaT
    if isinstance(x,str):
        s = x.strip()
        if s == "" or s.upper() in {"NA","N/A"}: return pd.NaT
        try: return parser.parse(s).date()
        except: return pd.NaT
    return pd.NaT

df["fecha_visita"] = df.get("fecha_visita").apply(safe_date)
df["fecha_visita"] = df["fecha_visita"].fillna(pd.to_datetime("2025-01-01").date())

# Días y gasto
df["dias_estadia"] = pd.to_numeric(df.get("dias_estadia"), errors="coerce").fillna(3).astype(int)
df["gasto_total"]  = pd.to_numeric(df.get("gasto_total"),  errors="coerce")
df["gasto_total"]  = df["gasto_total"].fillna(df.groupby("region")["gasto_total"].transform("mean"))
df["gasto_total"]  = df["gasto_total"].fillna(df["gasto_total"].median()).round(2)

# Extra: tipo_visitante
df["tipo_visitante"] = df["pais_origen"].apply(lambda x: "Nacional" if str(x).strip().lower() in {"perú","peru"} else "Extranjero")

df.to_csv(out_csv, index=False, encoding="utf-8")
print(f"✅ ETL: {orig} → {len(df)} filas. Guardado en: {out_csv}")


✅ ETL: 3031 → 3000 filas. Guardado en: D:\TurismoDigitalB_E1\database\visitantes_clean.csv


C:\Users\YATO\AppData\Local\Temp\ipykernel_3912\1642865973.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda v: v.strip() if isinstance(v, str) else v)
